In [1]:
!pip install gensim nltk scikit-learn tensorflow seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 67.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
df = pd.read_csv("alldata_1_for_kaggle.csv", encoding='latin-1')
df.head()

,Unnamed: 0,0,a
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...


In [9]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

df['clean_text'] = df['a'].apply(clean_text)

In [11]:
print(f"Number of unique values in column 'a': {df['a'].nunique()}")
print("\nTop 20 most frequent values in column 'a':")
display(df['a'].value_counts().head(20))

Number of unique values in column 'a': 977

Top 20 most frequent values in column 'a':


a
pancreatic cancer is a devastating malignancy with a 5year relative survival rate of only  dependenton the geographical location surveyed [] with these statistics exhibiting only modest improvementover the last four decades [] the median survival postdiagnosis ranges from just  months forlocally advanced disease and  months for metastatic disease   it was estimated by the world healthanisation that pancreatic cancer is currently the 7th leading cause of cancerrelated death being responsible for over  deaths worldwide in    with incidence increasing pancreatic cancerhas been predicted to be the third leading cause of cancerrelated death in the european union by   and the second leading cause of cancerrelated death in the united states of america and germanyby   several factors contribute to the poor survival of pancreatic cancer patients a current lack of reliablediagnostic markers that would enable early screening   coupled with largely nonspecific symptoms ofdisease results in over  of patients presenting with metastatic disease at diagnosis   this subgroupof patients have limited therapeutic options and are thus typically administered palliative chemotherapyaimed at prolonging survival and reducing symptoms during endoflife care [] moreover whilstapproximately  of patients present with localised disease that is eligible for potentially curativesurgery   disease recurs in over  of patients postresection   ultimately these factorsculminate in more than  of patients diagnosed with pancreatic cancer succumbing to disease  these harrowing statistics highlight that despite research efforts there remains a lack of understandingof the pathogenesis of disease which in turn limits the development of new therapeuticsreceived  march revised  july accepted  august version of record published august   the authors this is an open access  published by portland press limited on behalf of the biochemical society and distributed under the creative commonsattribution license  cc byncnd 0cclinical science   101042cs20191211pancreatic ductal adenocarcinomapancreatic ductal adenocarcinoma is the predominant pancreaticmalignancypancreatic cancer can arise from either the endocrine or the exocrine region of the pancreas tumours arising fromthe exocrine compartment are termed pancreatic ductal adenocarcinoma pdac and account for over  of allpancreatic cancers  pdac develops via a stepwise progression from normal tissue through to invasive lesions which is associated withdistinct morphological characteristics [] it has been proposed that this process begins with a phenomenontermed acinartoductal metaplasia adm which is a normal homeostatic mechanism whereby acinar cells transdifferentiate into a ductal phenotype in response to particular stimuli   however if compounded by an oncogenichit cells are pushed towards a pathogenic phenotype that develops into pancreatic intraepithelial neoplasia panin[] disease progresses through preinvasive stages termed panin1a panin1b panin2 and panin3 priorto invasive pdac   this progression is associated with increasing nuclear atypia whereby the nuclei are no longerpositioned basally and loss of normal architecture as cells become more papillary in nature with panin3 lesionsdemonstrating increased mitoses   as disease progresses to pdac cells become invasive and breach the basement membrane growing through the extracellular matrix and metastasising to distant ans figure less common precursor lesions include intraductal papillary mucinous neoplasms ipmns and mucinous cysticneoplasms mcns that also develop through multistep processes   whilst they share some common featureseach lesion is morphologically and genetically distinct in contrast with panins that form within small ducts ipmnsdevelop within the primary or secondary branches of the main pancreatic duct whilst mcns lack ductal involvement an ammatory tumour microenvironment contributes to pdacpathogenesisan archetypal feature of pdac is the develop

In [21]:
from sklearn.preprocessing import LabelEncoder

X = df['clean_text']
y = df['0']

# Encode target labels to numerical values
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Use the encoded labels for train_test_split
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [17]:
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()


In [18]:
X_train_tfidf = X_train_tfidf.reshape(X_train_tfidf.shape[0], 1, X_train_tfidf.shape[1])
X_test_tfidf = X_test_tfidf.reshape(X_test_tfidf.shape[0], 1, X_test_tfidf.shape[1])


In [22]:
num_classes = len(le.classes_)

model_tfidf = Sequential([
    SimpleRNN(64, input_shape=(1, X_train_tfidf.shape[2])),
    Dropout(0.3),
    Dense(num_classes, activation='softmax') # Changed for multi-class classification
])

model_tfidf.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Changed loss for multi-class integer labels
model_tfidf.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 64)             │       324,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 324,355 (1.24 MB)

 Trainable params: 324,355 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history_tfidf = model_tfidf.fit(
    X_train_tfidf, y_train_encoded,
    epochs=30,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.4230 - loss: 1.0713 - val_accuracy: 0.5752 - val_loss: 0.9726
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6465 - loss: 0.9181 - val_accuracy: 0.7168 - val_loss: 0.8711
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8477 - loss: 0.7718 - val_accuracy: 0.8230 - val_loss: 0.7700
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9182 - loss: 0.6335 - val_accuracy: 0.8142 - val_loss: 0.6709
Epoch 5/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9242 - loss: 0.5184 - val_accuracy: 0.8407 - val_loss: 0.5956
Epoch 6/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9384 - loss: 0.4472 - val_accuracy: 0.8319 - val_loss: 0.5325
Epoch 7/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9440 - loss: 0.3713 - val_accuracy: 0.8407 - val_loss: 0.4838
Epoch 8/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9252 - loss: 0.3211 - val_accuracy: 0.8407 - v

In [47]:
y_pred_probs = model_tfidf.predict(X_test_tfidf)
y_pred = np.argmax(y_pred_probs, axis=1)

acc_tfidf = accuracy_score(y_test_encoded, y_pred)
f1_tfidf = f1_score(y_test_encoded, y_pred, average='weighted')

print("Accuracy:", acc_tfidf)
print("F1 Score:", f1_tfidf)
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Accuracy: 0.8546099290780141
F1 Score: 0.8541541578160601
                precision    recall  f1-score   support

  Colon_Cancer       0.79      0.72      0.75        83
   Lung_Cancer       0.99      0.97      0.98        87
Thyroid_Cancer       0.80      0.87      0.83       112

      accuracy                           0.85       282
     macro avg       0.86      0.85      0.85       282
  weighted avg       0.86      0.85      0.85       282



In [26]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)


In [27]:
sentences = [text.split() for text in X_train]

w2v_skipgram = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=2,
    sg=1  # Skip-gram
)


In [37]:
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))

for word, i in word_index.items():
    if word in w2v_skipgram.wv:
        embedding_matrix[i] = w2v_skipgram.wv[word]


In [31]:
model_skipgram = Sequential([
    Embedding(len(word_index)+1, embedding_dim, weights=[embedding_matrix],
              input_length=max_len, trainable=False),
    SimpleRNN(64),
    Dropout(0.3),
    Dense(num_classes, activation='softmax') # Changed for multi-class classification
])

model_skipgram.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Changed loss for multi-class integer labels
model_skipgram.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │    16,789,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,789,900 (64.05 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 16,789,900 (64.05 MB)

In [32]:
history_skip = model_skipgram.fit(
    X_train_pad, y_train_encoded,
    epochs=30,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.3220 - loss: 1.3548 - val_accuracy: 0.4513 - val_loss: 1.0444
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.5062 - loss: 1.0073 - val_accuracy: 0.5221 - val_loss: 0.9662
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.4869 - loss: 0.9954 - val_accuracy: 0.5398 - val_loss: 0.9272
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5766 - loss: 0.8898 - val_accuracy: 0.5841 - val_loss: 0.8704
Epoch 5/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6529 - loss: 0.7953 - val_accuracy: 0.6018 - val_loss: 0.8454
Epoch 6/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.6454 - loss: 0.7645 - val_accuracy: 0.5752 - val_loss: 0.8683
Epoch 7/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6811 - loss: 0.6950 - val_accuracy: 0.6195 - val_loss: 0.8225
Epoch 8/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.6896 - loss: 0.6939 - val_accuracy: 0.6372 - v

In [48]:
y_pred_probs = model_skipgram.predict(X_test_pad)
y_pred = np.argmax(y_pred_probs, axis=1)

acc_skip = accuracy_score(y_test_encoded, y_pred)
f1_skip = f1_score(y_test_encoded, y_pred, average='weighted')

print("Accuracy:", acc_skip)
print("F1 Score:", f1_skip)
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Accuracy: 0.7553191489361702
F1 Score: 0.7554497290760861
                precision    recall  f1-score   support

  Colon_Cancer       0.70      0.66      0.68        83
   Lung_Cancer       0.90      0.72      0.80        87
Thyroid_Cancer       0.71      0.85      0.78       112

      accuracy                           0.76       282
     macro avg       0.77      0.75      0.75       282
  weighted avg       0.77      0.76      0.76       282



In [34]:
w2v_cbow = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=2,
    sg=0  # CBOW
)


In [39]:
embedding_matrix_cbow = np.zeros((len(word_index)+1, embedding_dim))

for word, i in word_index.items():
    if word in w2v_cbow.wv:
        embedding_matrix_cbow[i] = w2v_cbow.wv[word]


In [50]:
from tensorflow.keras.layers import LSTM

model_cbow = Sequential([
    Embedding(len(word_index)+1, embedding_dim, weights=[embedding_matrix_cbow],
              input_length=max_len, trainable=False),
    SimpleRNN(64), # Reverted to SimpleRNN with original units
    Dropout(0.3), # Reverted dropout rate
    Dense(num_classes, activation='softmax')
])

model_cbow.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cbow.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │    16,789,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_6 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,789,900 (64.05 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 16,789,900 (64.05 MB)

In [51]:
history_cbow = model_cbow.fit(
    X_train_pad, y_train_encoded,
    epochs=30, # Reverted to original epochs to keep it consistent with the other SimpleRNN models
    batch_size=64, # Reverted to original batch size
    validation_split=0.1
)

Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - accuracy: 0.3859 - loss: 1.3738 - val_accuracy: 0.4425 - val_loss: 1.1179
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.4717 - loss: 1.0786 - val_accuracy: 0.4779 - val_loss: 1.0199
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.4925 - loss: 1.0256 - val_accuracy: 0.4602 - val_loss: 0.9979
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.5661 - loss: 0.9110 - val_accuracy: 0.5044 - val_loss: 0.9765
Epoch 5/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.5999 - loss: 0.8541 - val_accuracy: 0.5398 - val_loss: 0.9427
Epoch 6/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.6307 - loss: 0.8143 - val_accuracy: 0.5221 - val_loss: 0.9397
Epoch 7/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6606 - loss: 0.7588 - val_accuracy: 0.5664 - val_loss: 0.9154
Epoch 8/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6432 - loss: 0.7656 - val_accuracy: 0.5929 - v

In [52]:
y_pred_probs = model_cbow.predict(X_test_pad)
y_pred = np.argmax(y_pred_probs, axis=1)

acc_cbow = accuracy_score(y_test_encoded, y_pred)
f1_cbow = f1_score(y_test_encoded, y_pred, average='weighted')

print("Accuracy:", acc_cbow)
print("F1 Score:", f1_cbow)
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Accuracy: 0.7056737588652482
F1 Score: 0.7007032481767234
                precision    recall  f1-score   support

  Colon_Cancer       0.65      0.53      0.58        83
   Lung_Cancer       0.82      0.68      0.74        87
Thyroid_Cancer       0.68      0.86      0.76       112

      accuracy                           0.71       282
     macro avg       0.71      0.69      0.69       282
  weighted avg       0.71      0.71      0.70       282



In [53]:
results = pd.DataFrame({
    "Model": ["RNN + TFIDF", "RNN + Skipgram", "RNN + CBOW"],
    "Accuracy": [acc_tfidf, acc_skip, acc_cbow],
    "F1 Score": [f1_tfidf, f1_skip, f1_cbow]
})

results

,Model,Accuracy,F1 Score
0,RNN + TFIDF,0.854610,0.854154
1,RNN + Skipgram,0.755319,0.755450
2,RNN + CBOW,0.705674,0.700703
